# Lielie valodu modeļi un rīku izsaukšana

## API klienta sagatavošana


In [ ]:
from openai import OpenAI
client = OpenAI(base_url="http://gauja.ailab.lv:30054/v1", api_key="dummy-key")

In [ ]:
# Debugging function to print messages
import json
from IPython.display import display, Markdown
def print_message(msg):
    if not isinstance(msg, dict):
      msg = msg.model_dump()
    print(f"Role: {msg.get('role', 'N/A')}")
    if 'content' in msg and msg['content']:
        display(Markdown(msg['content']))
    else:
      print(json.dumps(msg, indent=4, ensure_ascii=False))
    print("-" * 20)

## Laika ziņas

In [ ]:
def get_weather(place):
  return """"Līdzīgi kā aizvadītajā dienās, arī otrdien laiks būs pelēcīgs,
daudzviet mākoņi palaikam nesīs lietu.
Dienā saglabāsies lēns līdz mērens dienvidu,
dienvidrietumu vējš, kas dienas vidu piekrastē iegriezīsies no rietumiem,
un gaiss iesils līdz +6…+11°C."""

def get_location():
  return "Rīga"

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Funkcija atgriež šīs dienas laikapstākļu prognozi dotajai vietai.",
            "parameters": {
                "type": "object",
                "properties": {
                    "place": {
                        "type": "string",
                        "description": "Name of the palce"
                    }
                },
                "required": ["guess"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_location",
            "description": "Funkcija atgriež lietotāja atrašanās vietas nosaukumu.",
        },
    }
]

messages = []
msg = {"role": "user", "content": "Vai šodien ceļi būs slideni?"}
print_message(msg)
messages.append(msg)

while True:
    response = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    print_message(response.choices[0].message)

    if response.choices[0].message.tool_calls:
        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
            if function_name == "get_weather":
                arguments = json.loads(tool_call.function.arguments)
                place = arguments.get("place")
                reply = {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "content": get_weather(place),
                }
            elif function_name == "get_location":
                reply = {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "content": get_location(),
                }
            else:
                reply = {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "content": "Unknown function: " + function_name,
                }
            print_message(reply)
            messages.append(reply)
    else:
        break




Role: user


Vai šodien ceļi būs slideni?

--------------------
Role: assistant
{
    "content": null,
    "refusal": null,
    "role": "assistant",
    "annotations": null,
    "audio": null,
    "function_call": null,
    "tool_calls": [
        {
            "id": "chatcmpl-tool-803e437e8dab4c1e9bada6838ee835f6",
            "function": {
                "arguments": "{}",
                "name": "get_location"
            },
            "type": "function"
        }
    ],
    "reasoning_content": "The user asks in Latvian: \"Vai šodien ceļi būs slideni?\" meaning \"Will the roads be icy today?\" We need to provide an answer about road conditions. Probably need weather info for the user's location. Use get_location to get location name then get_weather for that place.\n\nWe need to call get_location first."
}
--------------------
Role: tool


Rīga

--------------------
Role: assistant
{
    "content": null,
    "refusal": null,
    "role": "assistant",
    "annotations": null,
    "audio": null,
    "function_call": null,
    "tool_calls": [
        {
            "id": "chatcmpl-tool-013c06b408b84d27865ea008e3ca5cec",
            "function": {
                "arguments": "{\"place\": \"R\\u012bga\"}",
                "name": "get_weather"
            },
            "type": "function"
        }
    ],
    "reasoning_content": "The user asks in Latvian: \"Vai šodien ceļi būs slideni?\" → \"Will the roads be icy today?\" Need to get weather for location. We have location name from get_location: it returned Rīga. Need to get weather for Rīga. Use get_weather."
}
--------------------
Role: tool


"Līdzīgi kā aizvadītajā dienās, arī otrdien laiks būs pelēcīgs, 
daudzviet mākoņi palaikam nesīs lietu. 
Dienā saglabāsies lēns līdz mērens dienvidu, 
dienvidrietumu vējš, kas dienas vidu piekrastē iegriezīsies no rietumiem, 
un gaiss iesils līdz +6…+11°C.

--------------------
Role: assistant


Saskaņā ar šodienas prognozi Rīgā temperatūra būs no +6 °C līdz +11 °C, sagaidāms mākoņains laiks ar nokrišņiem un dienvidu‑dienvidrietumu vēju. Tā kā temperatūra nepārsniegs nulli, ceļi nav sagaidāmi kā slideni no ledus. 

Tomēr, ņemot vērā lietus iespējamību, uz ceļa var būt mitras un slidenas vietas, it īpaši, ja uz tās iekrīt riepu spīdums vai notiek neapzinātas dabas dažādības (piemēram, krusas no ledus malām). Tāpēc vadītājiem ieteicams:

* Braukt piesardzīgi, samazināt ātrumu pie kruģiem un pārejas posmiem.
* Saglabāt pietiekamu attālumu no priekšā braucošā transporta.
* Pārbaudīt riepu riepu stāvokli un, ja iespējams, izmantot riepu ķēdes vai pret-slīdes riepu maisiņus.

Kopumā reālu ledus veidošanos uz ceļiem šodien sagaidīt neviens nav jābaidās. 🚗💧

--------------------


## Lielie valodu modeļi spēlē Wordle

Sagatavoja

Sagatavoja funkciju, kas pārbauda Wordle minējumu pret mērķa vārdu.

In [ ]:
TARGET = "horse"

def check_wordle_guess(guess: str) -> str:
    """
    Checks a Wordle guess against a hardcoded target word and returns the result.

    Args:
        guess: The guessed word (string).

    Returns:
        A string representing the result of the guess, using 'G' for correct
        position, 'Y' for correct letter in wrong position, and 'B' for
        incorrect letter.
    """
    target_word = TARGET.lower()
    guess = guess.lower()
    if len(guess) != len(target_word):
        return "Invalid guess length"

    result = [""] * len(target_word)
    target_letters = list(target_word)
    guess_letters = list(guess)

    # First pass: Check for correct position (Green)
    for i in range(len(target_word)):
        if guess_letters[i] == target_letters[i]:
            result[i] = 'G'
            target_letters[i] = None  # Mark as used
            guess_letters[i] = None # Mark as used

    # Second pass: Check for correct letter in wrong position (Yellow) and incorrect letter (Black)
    for i in range(len(target_word)):
        if guess_letters[i] is not None:
            if guess_letters[i] in target_letters:
                result[i] = 'Y'
                target_letters[target_letters.index(guess_letters[i])] = None # Mark as used
            else:
                result[i] = 'B'

    return "".join(result)

# Example usage (for testing purposes)
print("Target word", TARGET)
print("world", check_wordle_guess("world"))
print("esroh", check_wordle_guess("esroh"))


Target word horse
world BGGBB
esroh YYGYY


Nodefinējam to kā rīku, ko padosim OpenAI API.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "check_wordle_guess",
            "description": "Checks a 5-letter word guess against the target Wordle word. Returns a 5-character string indicating the result of the guess, using 'G' (Green) for correct position, 'Y' (Yellow) for correct letter in wrong position, and 'B' (Black) for incorrect letter.",
            "parameters": {
                "type": "object",
                "properties": {
                    "guess": {
                        "type": "string",
                        "description": "The 5-letter word guess."
                    }
                },
                "required": ["guess"],
            },
        },
    }
]

Izsaucam OpenAI API, lai tas uzmin vārdu.

In [ ]:
messages = []
msg = {"role": "user", "content": "Guess the word using check_wordle_guess."}
print_message(msg)
messages.append(msg)

while True:
    response = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    messages.append(response.choices[0].message)

    print_message(response.choices[0].message)

    if response.choices[0].message.tool_calls:
        tool_call  = response.choices[0].message.tool_calls[0]
        function_name = tool_call.function.name
        if function_name == "check_wordle_guess":
            reply = {}
            try:
                arguments = json.loads(tool_call.function.arguments)
                guess = arguments.get("guess")
                if guess and isinstance(guess, str):
                    tool_output = check_wordle_guess(guess)
                    reply = {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "content": tool_output,
                    }
                else:
                    reply = {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "content": "Error: Invalid guess format.",
                    }
            except json.JSONDecodeError:
                  reply = {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "content": "Error: Invalid JSON arguments.",
                }

            messages.append(reply)
            print_message(reply)
    else:
        break

print("Conversation finished.")

Role: user


Guess the word using check_wordle_guess.

--------------------
Role: assistant
{
    "content": null,
    "refusal": null,
    "role": "assistant",
    "annotations": null,
    "audio": null,
    "function_call": null,
    "tool_calls": [
        {
            "id": "chatcmpl-tool-0c1fc446e96743a288b7497c0ed33c12",
            "function": {
                "arguments": "{\"guess\": \"CRANE\"}",
                "name": "check_wordle_guess"
            },
            "type": "function"
        }
    ],
    "reasoning_content": "The user wants to guess the Wordle word using the tool. We need to interactively guess? There's no target word given. The tool likely uses a hidden target. We need to guess a word and get feedback, perhaps iteratively to narrow down. The user says \"Guess the word using check_wordle_guess.\" So we must attempt to guess the hidden word. We'll need to make guesses and use feedback to converge. Probably we can start with a common starting word like \"CRANE\". We'll call function."
}
--------------------
Role

BYBBG

--------------------
Role: assistant
{
    "content": null,
    "refusal": null,
    "role": "assistant",
    "annotations": null,
    "audio": null,
    "function_call": null,
    "tool_calls": [
        {
            "id": "chatcmpl-tool-40cedd36bd4e46e7a5461c1e2468ce93",
            "function": {
                "arguments": "{\"guess\":\"ROUGE\"}",
                "name": "check_wordle_guess"
            },
            "type": "function"
        }
    ],
    "reasoning_content": "We have a Wordle guess \"CRANE\" returned \"BYBBG\".\nInterpretation: positions: C B? Actually result: B Y B B G? Wait result string length 5: \"BYBBG\". \nPositions:\n1: B (C is not in word)\n2: Y (R is in word but wrong position)\n3: B (A not in word)\n4: B (N not in word)\n5: G (E correct position)\n\nSo target word has E at position5. Contains R somewhere but not position2. Does not contain C, A, N. Need to find 5-letter word with pattern _ _ _ _ E, contains R not at position2. Also no letters C, A, N.

...

--------------------
Role: assistant
{
    "content": null,
    "refusal": null,
    "role": "assistant",
    "annotations": null,
    "audio": null,
    "function_call": null,
    "tool_calls": [
        {
            "id": "chatcmpl-tool-d12f4dc2ef1c483c920abad99a249319",
            "function": {
                "arguments": "{\"guess\":\"FORTE\"}",
                "name": "check_wordle_guess"
            },
            "type": "function"
        }
    ],
    "reasoning_content": "We need to guess the hidden target word. We have two guesses and feedback.\n\nFeedback format: each position G=green correct letter, Y=yellow correct letter wrong position, B=black.\n\nFirst guess: CRANE -> feedback BYBBG\n\nPositions:\n1 C -> B (letter not in word)\n2 R -> Y (letter R is in word but not position 2)\n3 A -> B (A not in word)\n4 N -> B (N not in word)\n5 E -> G (E correct at position 5)\n\nSo we know word ends with E. Contains R somewhere not at position2. No C, A, N.\n\nSecond guess: ROUGE

BGGBG

--------------------
Role: assistant


The word is **HORSE**.

--------------------
Conversation finished.
